In [5]:
import torch, random, text_pretreatment
from torch import nn

# 一、读取长序列数据
"""
模型中的网络一次处理具有预定义长度(例如n个时间步)的一个小批量序列,
问题是如何随机生成一个小批量数据的特征和标签以供读取 --> 随机采样 与 顺序分区

随机采样:
在随机采样中，每个样本都是在原始的长序列上任意捕获的子序列。
在迭代过程中，来自两个相邻的、随机的、小批量中的子序列不一定在原始序列上相邻。
对于语言建模，目标是基于到目前为止我们看到的词元来预测下一个词元，因此标签是移位了一个词元的原始序列。

顺序分区:
在迭代过程中，除了对原始序列可以随机抽样外，还可以保证两个相邻的小批量中的子序列在原始序列上也是相邻的。
这种策略在基于小批量的迭代过程中保留了拆分的子序列的顺序，因此称为顺序分区。
"""

def get_random_batch_seq(corpus, batch_size, num_steps):
    """
    使用随机抽样生成一个小批量序列\n
    corpus : 语料库
    batch_size : 一个小批量中有多少个子序列样本
    num_steps : 每个序列预定义的时间步数
    """
    corpus = corpus[random.randint(0, num_steps - 1):] # 随机选择起始分区的偏移量,随机范围包括num_steps-1  减去1是因为需要考虑标签
    num_subseqs = (len(corpus) - 1) // num_steps # 整个批量中子序列的数量
    initial_indices = list(range(0, num_subseqs * num_steps, num_steps)) # 长度为num_step的每个子序列的起始索引
    random.shuffle(initial_indices) #  在随机抽样的迭代过程中,来自两个相邻的、随机的、小批量中的子序列不一定在原始序列上相邻
    num_batches = num_subseqs // batch_size # 整个批量可被分成的小批量的数量

    def get_seq(pos):
        """
        返回从pos位置开始的长度为num_steps的序列\n
        pos : 一个偏移量
        """
        return corpus[pos: pos + num_steps]

    for i in range(0, batch_size * num_batches, batch_size): # 迭代小批量
        initial_indices_per_batch = initial_indices[i : i+batch_size] # 在这里，initial_indices包含子序列的随机起始索引
        X = [get_seq(j) for j in initial_indices_per_batch]
        Y = [get_seq(j+1) for j in initial_indices_per_batch]
        yield torch.tensor(X), torch.tensor(Y) # 特征 和 对应的标签

def get_sequential_batch_seq(corpus, batch_size, num_steps):
    """
    使用顺序分区生成一个小批量子序列\n
    corpus : 语料库
    batch_size : 一个小批量中有多少个子序列样本
    num_steps : 每个序列预定义的时间步数
    """
    offset = random.randint(0,num_steps-1) # 用随机偏移量划分序列
    num_tokens = ((len(corpus)-offset-1) // batch_size) * batch_size # token数
    Xs = torch.tensor(corpus[offset : offset + num_tokens])
    Ys = torch.tensor(corpus[offset+1 : offset + num_tokens + 1])
    Xs, Ys = Xs.reshape(batch_size, -1), Ys.reshape(batch_size, -1)
    num_batches = Xs.shape[1] // num_steps # 小批量的个数
    for i in range(0, num_batches*num_steps, num_steps):
        X = Xs[: ,i:i + num_steps] # 特征
        Y = Ys[: ,i:i + num_steps] # 标签
        yield X, Y

# 将上面的两个采样函数包装到一个类中，以便稍后可以将其用作数据迭代器
class SeqDataLoader:
    """
    加载序列数据的迭代器
    """
    def __init__(self, batch_size, num_steps, max_tokens, use_random_iter) -> None:
        if use_random_iter: 
            self.data_iter_fn = get_random_batch_seq
        else: 
            self.data_iter_fn = get_sequential_batch_seq
        self.corpus, self.vocab = text_pretreatment.load_time_machine_corpus(max_tokens)
        self.batch_size, self.num_steps = batch_size, num_steps

    def __iter__(self):
        return self.data_iter_fn(self.corpus, self.batch_size, self.num_steps)
    
# 同时返回数据迭代器和词表，因此可以与其他带有load_data前缀的函数（如load_data_fashion_mnist）类似地使用。
def load_time_machine_data(batch_size, num_steps, 
                           max_tokens=10000, use_random_iter=False):
    """
    返回时光机器数据集的迭代器和词表
    """
    data_iter = SeqDataLoader(batch_size, num_steps, max_tokens, use_random_iter)
    return data_iter, data_iter.vocab

X:  tensor([[14, 15, 16, 17, 18],
        [ 4,  5,  6,  7,  8]]) 
Y: tensor([[15, 16, 17, 18, 19],
        [ 5,  6,  7,  8,  9]])
X:  tensor([[19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28]]) 
Y: tensor([[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29]])
X:  tensor([[ 9, 10, 11, 12, 13],
        [29, 30, 31, 32, 33]]) 
Y: tensor([[10, 11, 12, 13, 14],
        [30, 31, 32, 33, 34]])
X:  tensor([[ 5,  6,  7,  8,  9],
        [19, 20, 21, 22, 23]]) 
Y: tensor([[ 6,  7,  8,  9, 10],
        [20, 21, 22, 23, 24]])
X:  tensor([[10, 11, 12, 13, 14],
        [24, 25, 26, 27, 28]]) 
Y: tensor([[11, 12, 13, 14, 15],
        [25, 26, 27, 28, 29]])
